In [1]:
from os import environ

environ['HF_HOME'] = '/Volumes/DATA/hf_home'
environ["HTTP_PROXY"] = "http://localhost:7890"
environ["HTTPS_PROXY"] = "http://localhost:7890"
environ["OPENAI_API_BASE"] = "http://10.1.3.28:8990/v1"
environ["OPENAI_API_KEY"] = "sk-"

from retrieval_qa_benchmark.models import *
from retrieval_qa_benchmark.datasets import *
from retrieval_qa_benchmark.transforms import *
from retrieval_qa_benchmark.protocols import *
from retrieval_qa_benchmark.utils.registry import REGISTRY
from retrieval_qa_benchmark.utils.profiler import PROFILER

print(str(REGISTRY))

Datasets:
	hotpot_qa
	mmlu
	wikihop
Transforms:
	base
	mcsa_prompt
	mcsa_fewshot_mmlu
	mcma_prompt
Models:
	remote-llm
	gpt35
	chatgpt35
Evaluators:
	mcma
	mcsa


/Users/mpsk/Library/Caches/pypoetry/virtualenvs/retrieval-qa-benchmark-OUwPJWBa-py3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/mpsk/Library/Caches/pypoetry/virtualenvs/retrieval-qa-benchmark-OUwPJWBa-py3.11/lib/python3.11/site-packages/pydantic/_internal/_fields.py:127: UserWarning: Field "model_name" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


In [2]:
import yaml

config = yaml.safe_load(open("../config/mmlu-llama2-remote.yaml"))
evaluator = REGISTRY.Evaluators["mcsa"].from_config(config)
print(evaluator.transform)

Converting dataset...: 100%|██████████| 324/324 [00:00<00:00, 38774.10it/s]

chain=[MultipleChoiceTransform(sep_chr='\n', prompt_prefix='The following are multiple choice questions (with answers)\n', prompt_suffix='Answer: ')]


In [3]:
evaluator.transform(evaluator.dataset[0])
evaluator.dataset.eval_set = evaluator.dataset.eval_set[:5]

In [4]:
matched = evaluator()

Evaluating: 100%|██████████| 5/5 [02:23<00:00, 28.74s/it]
2023-08-07 14:47:46.620 | INFO     | retrieval_qa_benchmark.protocols.base:__call__:62 - Evaluation finished! Executed Evaluator:<class 'retrieval_qa_benchmark.protocols.mcsa.MCSAEvaluator'> on Dataset:cais/mmlu.prehistory-test with Model:llama2-13b-chat. Accuracy: 40.00%


In [6]:
print(str(PROFILER))

	======== Profile Result ========
BaseTransform.chain           	Count:     330	AvgTime:    0.01 msBaseDataset.__getitem__       	Count:       1	AvgTime:    0.01 msTransformChain.__call__       	Count:       6	AvgTime:    0.20 msBaseModel.generate            	Count:       5	AvgTime:28741.18 ms


In [5]:
with open('results.jsonl', 'w') as f:
    f.write('\n'.join([r.model_dump_json() for r in matched[1]]))

In [ ]:
for dname, dset in REGISTRY.Datasets.items():
    print(f"Loading {dname}...")
    dset.build()

In [7]:
from retrieval_qa_benchmark.schema import QAPrediction
mismatched = [pred for pred in matched[1] if pred.matched]

In [8]:
with open('results.test.jsonl', 'w') as f:
    f.write('\n'.join([r.model_dump_json() for r in mismatched]))